In [4]:
import pandas as pd
from modeling import feature_engineering, closest, agg_function,agg_diff_fun, save_object, load_object, model_predict, save_csv
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold


In [2]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
metro= pd.read_csv('MetroStanice.csv')
df_train["training"]=1
df_test["training"]=0
df= pd.concat([df_train,df_test],sort=True).reset_index(drop=True)

mapping_features= pd.read_csv('feature_deduplication.csv')
mydict= dict(zip(mapping_features.original_feature, mapping_features.unique_feature))

#features and photos
df["len_features"] = df["features"].apply(len)
df["len_photos"] = df["photos"].apply(len)
df['features']= df['features'].apply(lambda x: [item.lower() for item in x]) 
df['features']=df['features'].apply(lambda x: [mydict[item] if item in mydict else item for item in x ])
df['features_txt']= df['features'].apply(lambda x: ' '.join(x))
#pocet features
#

#timefromepoch
import datetime
#df['tstamp'] =df['created'].apply(lambda x:  datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())
#Datumove
#df["date"]=  df['created'].str[:7]
df['created']=pd.to_datetime(df["created"])
#df['day'] = df['created'].dt.day
#df['days']=(df["created"]-min(df["created"])).apply(lambda x: x.days)
df['dayofweek'] = df['created'].dt.dayofweek
#cylicke
df['hr'] = df['created'].dt.hour
df['mnth'] = df['created'].dt.month
df['nwords_description']= df['description'].apply(lambda x: x.count(' ')+1)
df['price_digits']=df['price'].apply(lambda x: len(str(x)))
df['price'] = np.log(df['price'])

# GEO
coordinates= df[["latitude","longitude"]].values
coordinates_tuple=tuple((x[0],x[1]) for x in coordinates)
adresy=revgc.search(coordinates_tuple)
df=pd.concat([df,pd.DataFrame(adresy)],axis=1)


#label encoding
event_dictionary ={'low' : 1, 'medium' : 2, 'high' : 3} 
df['interest_integer'] =df['interest_level'].map(event_dictionary) 

#interakcie zakladna intuicia
df['bed+bath']=df['bedrooms']+df['bathrooms']
df['price/bed']=df['price']/(df['bedrooms']+1e-16)
df['price/bed+bath']=df['price']/(df['bed+bath']+1e-16)

training='training'
label='interest_integer'

metro=metro.loc[:,['lat','lon']]
metro=metro.to_dict('records')
dfmetro=df.loc[:,['latitude','longitude']]
dfmetro=dfmetro.to_dict('records')
x=[]
for i in dfmetro:
    x.append(closest(metro, i))
df['metro']=x


#drop columns
df=df.drop(columns=['photos','created','listing_id','lat','lon','latitude','longitude'])


Loading formatted geocoded file...


In [3]:
clf=xgb.XGBClassifier(n_jobs=-1,n_estimators=100,random_state=0,eval_metric='mlogloss',tree_method='gpu_hist')
#clf=xgb.XGBClassifier(n_jobs=-1,n_estimators=100,random_state=0,eval_metric='mlogloss')
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[1,2,3,4,5]
num_features=50
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf=clf)
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_thesis')

2020-05-23 18:41:03.707702


100%|██████████| 11/11 [01:11<00:00,  6.49s/it]


after numeric feature selection:  ['bathrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'metro'] 0.632116124950968


  0%|          | 0/7 [00:00<?, ?it/s]

after TOP N features:  ['bathrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'metro', 'elevator', 'hardwood', 'cats allowed', 'concierge', 'dish washer', 'no fee', 'fitness center', 'pre war', 'roof', 'new construction', 'simplex', 'fireplace', 'garage', 'reduced fee', 'furnished', 'multi level', 'live in super', 'washer & dryer', 'on-site laundry'] 0.6137549197086272


...

3 <function stemming_tokenizer at 0x7faf7a483440> 1
3 <function stemming_tokenizer at 0x7faf7a483440> 2
3 <function stemming_tokenizer at 0x7faf7a483440> 3
3 <function stemming_tokenizer at 0x7faf7a483440> 4
3 <function stemming_tokenizer at 0x7faf7a483440> 5


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


3 <modeling.LemmaTokenizer object at 0x7faf4d254690> 1
3 <modeling.LemmaTokenizer object at 0x7faf4d254690> 2
3 <modeling.LemmaTokenizer object at 0x7faf4d254690> 3
3 <modeling.LemmaTokenizer object at 0x7faf4d254690> 4
3 <modeling.LemmaTokenizer object at 0x7faf4d254690> 5
3 None 1
3 None 2
3 None 3
3 None 4


100%|██████████| 3/3 [28:15<00:00, 565.24s/it]

3 None 5
after text feature tokenization plus PCA:  ['bathrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'metro', 'elevator', 'hardwood', 'cats allowed', 'concierge', 'dish washer', 'no fee', 'fitness center', 'pre war', 'roof', 'new construction', 'simplex', 'fireplace', 'garage', 'reduced fee', 'furnished', 'multi level', 'live in super', 'washer & dryer', 'on-site laundry', 'mean_bathrooms_by_building_id', 'median_bathrooms_by_building_id', 'sum_bathrooms_by_building_id', 'std_bathrooms_by_building_id', 'bathrooms-mean_bathrooms_by_building_id', 'bathrooms-median_bathrooms_by_building_id', 'mean_bedrooms_by_building_id', 'median_bedrooms_by_building_id', 'sum_bedrooms_by_building_id', 'std_bedrooms_by_building_id', 'bedrooms-mean_bedrooms_by_building_id', 'bedrooms-median_bedrooms_by_building_id', 'mean_price_by_building_id', 'median_price_by_building_id', 'sum_price_by_building_id', 'std_price_by_building_id'

In [5]:
clf=xgb.XGBClassifier(n_jobs=-1,n_estimators=100,random_state=0,tree_method='gpu_hist')
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[2,3,4,5]
num_features=25
df_out,columns,good_encoders=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf=clf)
save_object([df_out,columns,clf,good_encoders],clf.__class__.__name__+'_best_grid_params')

2020-04-25 17:14:52.856601


100%|██████████| 11/11 [01:11<00:00,  6.52s/it]


after numeric features selection:  ['bathrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'metro'] 0.632116124950968


  0%|          | 0/7 [00:00<?, ?it/s]

after TOP N feature:  ['bathrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'metro', 'elevator', 'cats allowed', 'hardwood floors', 'doorman', 'dishwasher', 'laundry in building', 'no fee', 'fitness center', 'pre-war', 'outdoor space', 'dining room', 'swimming pool', 'prewar', 'wheelchair access'] 0.6163180621682092
bathrooms building_id 0.6163180621682092 ['bathrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'metro', 'elevator', 'cats allowed', 'hardwood floors', 'doorman', 'dishwasher', 'laundry in building', 'no fee', 'fitness center', 'pre-war', 'outdoor space', 'dining room', 'swimming pool', 'prewar', 'wheelchair access']
bedrooms building_id 0.5977908427560236 ['bathrooms', 'price', 'len_features', 'len_photos', 'dayofweek', 'hr', 'mnth', 'nwords_description', 'bed+bath', 'price/bed', 'metro', 'elevator', 'cats allowed', 'hardwood floo

...

2 <function stemming_tokenizer at 0x7f0cb0190b90> 2
2 <function stemming_tokenizer at 0x7f0cb0190b90> 3
0.5525583392085675
2 <function stemming_tokenizer at 0x7f0cb0190b90> 4
2 <function stemming_tokenizer at 0x7f0cb0190b90> 5


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


2 <modeling.LemmaTokenizer object at 0x7f0cbdcbb450> 2
2 <modeling.LemmaTokenizer object at 0x7f0cbdcbb450> 3
2 <modeling.LemmaTokenizer object at 0x7f0cbdcbb450> 4
2 <modeling.LemmaTokenizer object at 0x7f0cbdcbb450> 5
2 None 2
2 None 3
2 None 4


100%|██████████| 2/2 [20:37<00:00, 618.79s/it]

2 None 5
